In [30]:
import pandas as pd
import numpy as np
from datetime import datetime
FMT = '%d,%H:%M:%S'

In [32]:
# Read all-word data (.csv)
d_allwork_df = pd.read_csv(r'Data/d_all_work.csv')
s_allwork_df = pd.read_csv(r'Data/s_all_work.csv')
d_event_df = pd.read_csv(r'Data/d_events.csv')
s_event_df = pd.read_csv(r'Data/s_events.csv')

#d_allwork_df.head(1)
#s_allwork_df.head(1)
#d_event_df.head(1)
#s_event_df.head(1)

def get_names(df):
    df = df['Student Name 1']
    n_list = []
    for name in df:
        if name not in n_list:
            n_list.append(name)
    return n_list

s_names = get_names(s_allwork_df)
d_names = get_names(d_allwork_df)

# print(s_names[:10])
# print(d_names[:10])

# Step 1.6
1. Time spends on page 1.6 on the first day; The rest of time spend on page 1.6
2. Number of revisiting before first feedback (after arriving on page 1.6, go back to previous pages)
Number of revisiting after first feedback

In [49]:
def step_161(df_161, names):
    duration_df = pd.DataFrame()
    idx = 0
    for name in names:
        df = df_161.loc[df_161['Student Name 1'] == name][['Server Timestamp', 'Event']]
        # print(df)
        time_list = []
        duration = {'Student Name 1': name}
        counter = 0 # 0 means need enter, 1 means need exit
        if len(df) < 2:
            print('<', name, '> has less than 2 entries in [1.6]') 
            duration['Time1.6_Before'] = -2
            duration['Time1.6_After'] = -2
        else:
            for __, entry in df.iterrows():
                time_slice = str(entry[0]).split(' ')
                day = time_slice[2]
                time = time_slice[-1][2:]
                day_time = str(day) + ',' + str(time)
                if counter == 0 and str(entry[1]) == 'nodeEntered':
                    time_list.append(day_time)
                    counter = 1
                elif counter == 1 and str(entry[1]) == 'nodeExited':
                    time_list.append(day_time)
                    counter = 0
                else:
                    continue
            if len(time_list) % 2 != 0:
                time_list = time_list[0:-1]
            duration['Time1.6_Before'] = (datetime.strptime(time_list[1], FMT) - datetime.strptime(time_list[0], FMT)).seconds
            if len(time_list) == 2:
                duration['Time1.6_After'] = -1
            else:
                duration['Time1.6_After'] = 0
                for i in range(2, len(time_list), 2):
                    duration['Time1.6_After'] += (datetime.strptime(time_list[i+1], FMT) - datetime.strptime(time_list[i], FMT)).seconds
        duration_df = pd.concat([duration_df, pd.DataFrame(duration, index=[idx])], ignore_index=True)
        idx += 1
    return duration_df
    
def step_162(df_162, names):
    revisited_df = pd.DataFrame()
    idx = 0
    for name in names:
        revisited_dict = {'Student Name 1': name}
        df = df_162.loc[df_162['Student Name 1'] == name]
        rev_steps = ['1.1', '1.2', '1.3', '1.4', '1.5']
        step_dict = {}
        for __, row in df.iterrows():
            #print(row['Server Timestamp'])
            day = row['Server Timestamp'].split(' ')[2]
            if day not in step_dict:
                step_dict[day] = [str(row['Step Title'])[:3]]
            else:
                step_dict[day].append(str(row['Step Title'])[:3])
        counter = 0
        for day, steps in step_dict.items():
            indices = [i for i, x in enumerate(steps) if x == '1.6']
            # print(indices)
            if counter == 0:
                attr_name = 'Revisit1.6_Before'
                counter += 1 
            else:
                attr_name = 'Revisit1.6_After'
            revisited_dict[attr_name] = 0
            if len(indices) == 0:
                continue
            else:
                for i in indices:
                    if i < len(steps) - 1:
                        if str(steps[i + 1]) in rev_steps:
                            revisited_dict[attr_name] += 1
        # print(revisited_dict)
        revisited_df = pd.concat([revisited_df, pd.DataFrame(revisited_dict, index=[idx])], ignore_index=True)
        idx += 1
    # print(revisited_df)
    return revisited_df

def rw_161(df, names, output):
    df_161 = df.loc[df['Step Title'].str.contains('1.6', na=False)]
    df_161 = step_161(df_161, names)
    #print(step161(df_161))
    for idx, entry in df_161.iterrows():
        if entry[1] > 10000:
            df_161.loc[idx, 'Time1.6_Before'] = -2
    # path_name = 'Result/' + output
    # df_161.to_csv(path_name)
    # print('Write to ' + path_name)
    return df_161
    
def rw_162(df, names, output):
    df_162 = df[['Student Name 1', 'Server Timestamp', 'Step Title']]
    df_162 = step_162(df_162, names)
    # path_name = 'Result/' + output
    # df_162.to_csv(path_name)
    # print('Write to ' + path_name)
    return df_162


s_161 = rw_161(s_event_df, s_names, 's_161.csv')
d_161 = rw_161(d_event_df, d_names, 'd_161.csv')
print('-' * 25)
s_162 = rw_162(s_event_df, s_names, 's_162.csv')
d_162 = rw_162(d_event_df, d_names, 'd_162.csv')

Write to Result/s_161.csv
Write to Result/d_161.csv
-------------------------
Write to Result/s_162.csv
Write to Result/d_162.csv


# Step 1.7
1. Time spends on page 1.7 on the first day; The rest of time spend on page 1.7
2. Number of revisiting before first feedback (after arriving on page 1.7, go back to previous pages)
Number of revisiting after first feedback

In [58]:
FMT = '%d,%H:%M:%S'

def step_171(df_171, names):
    duration_df = pd.DataFrame()
    idx = 0
    for name in names:
        df = df_171.loc[df_171['Student Name 1'] == name][['Server Timestamp', 'Event']]
        # print(df)
        time_list = []
        duration = {'Student Name 1': name}
        counter = 0 # 0 means need enter, 1 means need exit
        if len(df) < 2:
            print('<', name, '> has less than 2 entries in [1.7]') 
            duration['Time1.7_Before'] = -2
            duration['Time1.7_After'] = -2
        else:
            for __, entry in df.iterrows():
                time_slice = str(entry[0]).split(' ')
                day = time_slice[2]
                time = time_slice[-1][2:]
                day_time = str(day) + ',' + str(time)
                if counter == 0 and str(entry[1]) == 'nodeEntered':
                    time_list.append(day_time)
                    counter = 1
                elif counter == 1 and str(entry[1]) == 'nodeExited':
                    time_list.append(day_time)
                    counter = 0
                else:
                    continue
            if len(time_list) % 2 != 0:
                time_list = time_list[0:-1]
            duration['Time1.7_Before'] = (datetime.strptime(time_list[1], FMT) - datetime.strptime(time_list[0], FMT)).seconds
            if len(time_list) == 2:
                duration['Time1.7_After'] = -1
            else:
                duration['Time1.7_After'] = 0
                for i in range(2, len(time_list), 2):
                    duration['Time1.7_After'] += (datetime.strptime(time_list[i+1], FMT) - datetime.strptime(time_list[i], FMT)).seconds
        duration_df = pd.concat([duration_df, pd.DataFrame(duration, index=[idx])], ignore_index=True)
        idx += 1
    return duration_df

def step_172(df_172, names):
    revisited_df = pd.DataFrame()
    idx = 0
    for name in names:
        revisited_dict = {'Student Name 1': name}
        df = df_172.loc[df_172['Student Name 1'] == name]
        rev_steps = ['1.1', '1.2', '1.3', '1.4', '1.5', '1.6']
        step_dict = {}
        for __, row in df.iterrows():
            #print(row['Server Timestamp'])
            day = row['Server Timestamp'].split(' ')[2]
            if day not in step_dict:
                step_dict[day] = [str(row['Step Title'])[:3]]
            else:
                step_dict[day].append(str(row['Step Title'])[:3])
        counter = 0
        for day, steps in step_dict.items():
            indices = [i for i, x in enumerate(steps) if x == '1.6']
            # print(indices)
            if counter == 0:
                attr_name = 'Revisit1.7_Before'
                counter += 1 
            else:
                attr_name = 'Revisit1.7_After'
            revisited_dict[attr_name] = 0
            if len(indices) == 0:
                continue
            else:
                for i in indices:
                    if i < len(steps) - 1:
                        if str(steps[i + 1]) in rev_steps:
                            revisited_dict[attr_name] += 1
        # print(revisited_dict)
        revisited_df = pd.concat([revisited_df, pd.DataFrame(revisited_dict, index=[idx])], ignore_index=True)
        idx += 1
    # print(revisited_df)
    return revisited_df

def rw_171(df, names, output):
    df_171 = df.loc[df['Step Title'].str.contains('1.7', na=False)]
    df_171 = step_171(df_171, names)
    #print(step161(df_171))
    for idx, entry in df_171.iterrows():
        if entry[1] > 10000:
            df_171.loc[idx, 'Time1.7_Before'] = -2
    # path_name = 'Result/' + output
    # df_171.to_csv(path_name)
    # print('Write to ' + path_name)
    return df_171

def rw_172(df, names, output):
    df_172 = df[['Student Name 1', 'Server Timestamp', 'Step Title']]
    df_172 = step_172(df_172, names)
    # path_name = 'Result/' + output
    # df_172.to_csv(path_name)
    # print('Write to ' + path_name)
    return df_172

s_171 = rw_171(s_event_df, s_names, 's_171.csv')
d_171 = rw_171(d_event_df, d_names, 'd_171.csv')
print('-' * 25)
s_172 = rw_172(s_event_df, s_names, 's_172.csv')
d_172 = rw_172(d_event_df, d_names, 'd_172.csv')

Write to Result/s_171.csv
Write to Result/d_171.csv
-------------------------
Write to Result/s_172.csv
Write to Result/d_172.csv


# Step 1.8
1. Time spends on page 1.8 on the first day; The rest of time spend on page 1.8
2. Number of revisiting before first feedback (after arriving on page 1.8, go back to previous pages)
Number of revisiting after first feedback

In [59]:
FMT = '%d,%H:%M:%S'

def step_181(df_181, names):
    duration_df = pd.DataFrame()
    idx = 0
    for name in names:
        df = df_181.loc[df_181['Student Name 1'] == name][['Server Timestamp', 'Event']]
        # print(df)
        time_list = []
        duration = {'Student Name 1': name}
        counter = 0 # 0 means need enter, 1 means need exit
        if len(df) < 2:
            print('<', name, '> has less than 2 entries in [1.8]') 
            duration['Time1.8_Before'] = -2
            duration['Time1.8_After'] = -2
        else:
            for __, entry in df.iterrows():
                time_slice = str(entry[0]).split(' ')
                day = time_slice[2]
                time = time_slice[-1][2:]
                day_time = str(day) + ',' + str(time)
                if counter == 0 and str(entry[1]) == 'nodeEntered':
                    time_list.append(day_time)
                    counter = 1
                elif counter == 1 and str(entry[1]) == 'nodeExited':
                    time_list.append(day_time)
                    counter = 0
                else:
                    continue
            if len(time_list) % 2 != 0:
                time_list = time_list[0:-1]
            duration['Time1.8_Before'] = (datetime.strptime(time_list[1], FMT) - datetime.strptime(time_list[0], FMT)).seconds
            if len(time_list) == 2:
                duration['Time1.8_After'] = -1
            else:
                duration['Time1.8_After'] = 0
                for i in range(2, len(time_list), 2):
                    duration['Time1.8_After'] += (datetime.strptime(time_list[i+1], FMT) - datetime.strptime(time_list[i], FMT)).seconds
        duration_df = pd.concat([duration_df, pd.DataFrame(duration, index=[idx])], ignore_index=True)
        idx += 1
    return duration_df

def step_182(df_182, names):
    revisited_df = pd.DataFrame()
    idx = 0
    for name in names:
        revisited_dict = {'Student Name 1': name}
        df = df_182.loc[df_182['Student Name 1'] == name]
        rev_steps = ['1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7']
        step_dict = {}
        for __, row in df.iterrows():
            #print(row['Server Timestamp'])
            day = row['Server Timestamp'].split(' ')[2]
            if day not in step_dict:
                step_dict[day] = [str(row['Step Title'])[:3]]
            else:
                step_dict[day].append(str(row['Step Title'])[:3])
        counter = 0
        for day, steps in step_dict.items():
            indices = [i for i, x in enumerate(steps) if x == '1.6']
            # print(indices)
            if counter == 0:
                attr_name = 'Revisit1.8_Before'
                counter += 1 
            else:
                attr_name = 'Revisit1.8_After'
            revisited_dict[attr_name] = 0
            if len(indices) == 0:
                continue
            else:
                for i in indices:
                    if i < len(steps) - 1:
                        if str(steps[i + 1]) in rev_steps:
                            revisited_dict[attr_name] += 1
        # print(revisited_dict)
        revisited_df = pd.concat([revisited_df, pd.DataFrame(revisited_dict, index=[idx])], ignore_index=True)
        idx += 1
    # print(revisited_df)
    return revisited_df

def rw_181(df, names, output):
    df_181 = df.loc[df['Step Title'].str.contains('1.8', na=False)]
    df_181 = step_181(df_181, names)
    #print(step161(df_181))
    for idx, entry in df_181.iterrows():
        if entry[1] > 10000:
            df_181.loc[idx, 'Time1.8_Before'] = -2
    # path_name = 'Result/' + output
    # df_181.to_csv(path_name)
    # print('Write to ' + path_name)
    return df_181

def rw_182(df, names, output):
    df_182 = df[['Student Name 1', 'Server Timestamp', 'Step Title']]
    df_182 = step_182(df_182, names)
    # path_name = 'Result/' + output
    # df_182.to_csv(path_name)
    # print('Write to ' + path_name)
    return df_182

s_181 = rw_181(s_event_df, s_names, 's_181.csv')
d_181 = rw_181(d_event_df, d_names, 'd_181.csv')
print('-' * 25)
s_182 = rw_182(s_event_df, s_names, 's_182.csv')
d_182 = rw_182(d_event_df, d_names, 'd_182.csv')

< guobin s > has less than 2 entries in [1.8]
Write to Result/s_181.csv
Write to Result/d_181.csv
-------------------------
Write to Result/s_182.csv
Write to Result/d_182.csv


In [70]:
# Merge stuff
s_dfs = [s_161, s_162, s_171, s_172, s_181, s_182]

s_16 = pd.merge(s_161, s_162, on='Student Name 1', how='outer')
s_17 = pd.merge(s_171, s_172, on='Student Name 1', how='outer')
s_18 = pd.merge(s_181, s_182, on='Student Name 1', how='outer')

s = pd.merge(s_16, s_17, on='Student Name 1', how='outer')
s = pd.merge(s, s_18, on='Student Name 1', how='outer')

s.to_csv('Result/s_all.csv')

# Merge stuff
d_16 = pd.merge(d_161, d_162, on='Student Name 1', how='outer')
d_17 = pd.merge(d_171, d_172, on='Student Name 1', how='outer')
d_18 = pd.merge(d_181, d_182, on='Student Name 1', how='outer')

d = pd.merge(d_16, d_17, on='Student Name 1', how='outer')
d = pd.merge(d, d_18, on='Student Name 1', how='outer')

d.to_csv('Result/d_all.csv')

# Combine
all = pd.concat([s, d], ignore_index=True)
all.to_csv('Result/all.csv')